| Name | Type (Shape) | Description |
| --- | --- | --- |
| time | float | time |
| x | array (2,) | $ \mathbf{x} $ |
| X | array (NX, NY) | X-meshgrid of velocity data|
| Y | array (NX, NY) | Y-meshgrid of velocity data|
| Interpolant_u | Interpolant object | Interpolant object for $ u(\mathbf{x}, t)  $ |
| Interpolant_v | Interpolant object | Interpolant object for $ v(\mathbf{x}, t)  $ |
| periodic | list (3,) | periodic[0]: periodicity in x <br /> periodic[1]: periodicity in y <br /> periodic[2]: periodicity in time|
| defined_domain | array (NX, NY) | points on the meshgrid where velocity field is defined |
| bool_unsteady | bool | specifies if velocity field is unsteady/steady |
| time_data | array(1,NT) | time of velocity data |
| delta | list (2,) | delta[0]: x-grid spacing of the velocity data <br /> delta[1]: y-grid spacing of the velocity data|

In [1]:
import sys, os

# get current directory
path = os.getcwd()

# get parent directory
parent_directory = os.path.sep.join(path.split(os.path.sep)[:-1])

# add utils folder to current working path in order to access the functions
sys.path.append(parent_directory+"/utils")

In [2]:
# Import numpy
import numpy as np

# Import package to solve ODE
from scipy.integrate import solve_ivp

# function which evalutes derivative of flow map
from ipynb.fs.defs.dFdt import dFdt

# function which computes particle velocity
from ipynb.fs.defs.velocity import velocity

In [3]:
def RK4_step(t, x1, dt, X, Y, Interpolant_u, Interpolant_v, periodic, bool_unsteady, time_data):
    
    # Compute x_prime at the beginning of the time-step by re-orienting and rescaling the vector field
    x_prime = velocity(t, x1, X, Y, Interpolant_u, Interpolant_v, periodic, bool_unsteady, time_data)
    
    # compute derivative
    k1 = dt * x_prime

    # Update position at the first midpoint.
    x2 = x1 + .5 * k1
     
    # Update time
    t = t+.5*dt
    
    # Compute x_prime at the first midpoint.
    x_prime = velocity(t, x2, X, Y, Interpolant_u, Interpolant_v, periodic, bool_unsteady, time_data)
    
    # compute derivative
    k2 = dt * x_prime

    # Update position at the second midpoint.
    x3 = x1 + .5 * k2
    
    # Update time
    t = t+.5*dt
    
    # Compute x_prime at the second midpoint.
    x_prime = velocity(t, x3, X, Y, Interpolant_u, Interpolant_v, periodic, bool_unsteady, time_data)
    
    # compute derivative
    k3 = dt * x_prime
    
    # Update position at the endpoint.
    x4 = x1 + k3
    
    # Update time
    t = t+dt
    
    # Compute derivative at the end of the time-step.
    x_prime = velocity(t, x4, X, Y, Interpolant_u, Interpolant_v, periodic, bool_unsteady, time_data) 
    
    # compute derivative
    k4 = dt * x_prime
    
    # Compute RK4 derivative
    y_prime_update = 1.0 / 6.0*(k1 + 2 * k2 + 2 * k3 + k4)
    
    # Integration y <-- y + y_prime*dt
    y_update = x1 + y_prime_update
    
    return y_update, y_prime_update/dt